In [1]:
import pandas as pd
import string
from num2words import num2words
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import PorterStemmer, WordNetLemmatizer

### Data Preprocessing

In [2]:
# Read the CSV file
phrasebank_data = pd.read_csv('../phrasebank_data.csv')

# Change currency and % symbols to words
phrasebank_data['headline_text'] = phrasebank_data['headline_text'].str.replace('\$', 'dollars ', regex=True)
phrasebank_data['headline_text'] = phrasebank_data['headline_text'].str.replace('\%', ' percent', regex=True)

# Convert 'Sentence' column to lowercase text.
text_lowercase = phrasebank_data['headline_text'].str.lower()

# Remove punctuation and symbols from the text
text_no_punct = text_lowercase.str.translate(str.maketrans('', '', string.punctuation))

# Tokenize the dataset
data_tokenized = text_no_punct.apply(word_tokenize)

# Remove stopwords from the dataset
stop_words = set(stopwords.words('english'))
data_tokenized = data_tokenized.apply(lambda x: [word for word in x if word not in stop_words])

# Perform stemming and lemmatization
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
data_processed = data_tokenized.apply(lambda x: [stemmer.stem(lemmatizer.lemmatize(word)) for word in x])

# Replace digits with words
for row in data_processed:
    for i in range(len(row)):
        if row[i].isdigit():
            row[i] = num2words(int(row[i]))
        elif row[i].replace('.', '', 1).isdigit():
            row[i] = num2words(float(row[i]))

# Add ngrams to the dataset
data_ngrams = data_processed.apply(lambda x: list(ngrams(x, 3)))

# Remove rows with empty lists of tuples 
data_nulls = data_ngrams.apply(lambda x: x if len(x) > 0 else None)

# Concatenate ngrams and sentiment column into a new dataframe
preprocessed_data = pd.concat([data_nulls, phrasebank_data['human_label']], axis=1)

# Remove null lists 
preprocessed_data = preprocessed_data[~preprocessed_data['headline_text'].apply(lambda x: isinstance(x, list) and not x)]

# Remove null values
preprocessed_data = preprocessed_data.dropna(subset=['headline_text'])

# Save preprocessed data into a new CSV file
preprocessed_data.to_csv("../1_Preprocessed_Data/phrasebank_preprocessed.csv", index=False)
preprocessed_data.head()

headline_text human_label
0  [(accord, gran, compani), (gran, compani, plan...     neutral
1  [(technopoli, plan, develop), (plan, develop, ...     neutral
2  [(intern, electron, industri), (electron, indu...    negative
3  [(new, product, plant), (product, plant, compa...    positive
4  [(accord, compani, updat), (compani, updat, st...    positive

In [3]:
null_values = preprocessed_data['headline_text'].isnull().sum()
print(null_values)

0
